In [60]:
import datetime
import os
import sys
import math
import pandas as pd
import numpy as np
from pprint import pprint
import json
import requests
import os
from pandas.io.json import json_normalize

In [61]:
initial_value = "oura_2020-08-19_AG"


#personal glyph node files
mn = str(initial_value) + "_oura_v1sumscores_node.csv"

#met_1min tag files
mt = str(initial_value) + "_oura_v1sumscores_tag.csv"

In [62]:
with open('oura_2020-08-19_AG.json') as json_data:
    dataJ = json.load(json_data)
    
sleep = json_normalize(dataJ, 'sleep')
dfs = pd.DataFrame(sleep)

readiness = json_normalize(dataJ, 'readiness')
dfr = pd.DataFrame(readiness)

activity = json_normalize(dataJ, 'activity')
dfa = pd.DataFrame(activity)

In [63]:
df = dfa.merge(dfr, on='summary_date')

In [64]:
df = df.merge(dfs, on = 'summary_date')
df.rename( columns={'Unnamed: 0':'index'}, inplace=True )
df.head()

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_disturbances,score_efficiency,score_latency,score_rem,score_total,temperature_delta,temperature_deviation,temperature_trend_deviation,timezone_y,total_y
0,1.59375,634,3202,1111111111111111134322222222222222222222232222...,10257,2020-07-01T03:59:59-07:00,2020-06-30T04:00:00-07:00,0,909,2,...,49,65,89,8,58,-0.13,-0.13,NaN,-420,20640
1,1.34375,352,2787,1111111111111111111121111111111111111111223322...,5720,2020-07-02T03:59:59-07:00,2020-07-01T04:00:00-07:00,0,606,1,...,44,90,97,11,47,-0.10,-0.10,0.00,-420,17880
2,1.43750,448,2959,1111111111111111111111111112122233222333223334...,7030,2020-07-03T03:59:59-07:00,2020-07-02T04:00:00-07:00,0,580,0,...,49,81,97,27,63,-0.01,-0.01,0.02,-420,21630
3,1.75000,1077,3608,1111111111111111221111111111231113432234334334...,16495,2020-07-04T03:59:59-07:00,2020-07-03T04:00:00-07:00,16,489,1,...,46,88,67,60,71,0.03,0.03,0.06,-420,23700
4,1.62500,807,3312,1111111111111111111121111111211111111111111122...,13408,2020-07-05T03:59:59-07:00,2020-07-04T04:00:00-07:00,2,481,0,...,52,88,72,53,71,-0.29,-0.29,-0.03,-420,23730


In [65]:
df.shape

(50, 81)

In [66]:
#df.to_csv('A1oura_data.csv')

In [67]:
#df = pd.read_csv('A1oura_data.csv')

In [68]:
mylist = df.columns.values.tolist()
print(mylist)

['average_met', 'cal_active', 'cal_total', 'class_5min', 'daily_movement', 'day_end', 'day_start', 'high', 'inactive', 'inactivity_alerts', 'low', 'medium', 'met_1min', 'met_min_high', 'met_min_inactive', 'met_min_low', 'met_min_medium', 'non_wear', 'rest', 'score_x', 'score_meet_daily_targets', 'score_move_every_hour', 'score_recovery_time', 'score_stay_active', 'score_training_frequency', 'score_training_volume', 'steps', 'summary_date', 'target_calories', 'target_km', 'target_miles', 'timezone_x', 'to_target_km', 'to_target_miles', 'total_x', 'period_id_x', 'score_y', 'score_activity_balance', 'score_hrv_balance', 'score_previous_day', 'score_previous_night', 'score_recovery_index', 'score_resting_hr', 'score_sleep_balance', 'score_temperature', 'awake', 'bedtime_end', 'bedtime_end_delta', 'bedtime_start', 'bedtime_start_delta', 'breath_average', 'deep', 'duration', 'efficiency', 'hr_5min', 'hr_average', 'hr_lowest', 'hypnogram_5min', 'is_longest', 'light', 'midpoint_at_delta', 'mid

In [69]:
df['score_sum'] = df['score'] + df['score_x'] + df['score_y']
df1 = df.sort_values('score_sum')
#print(df1['score_sum'])

In [70]:
#df['score_sum'] = np.where(df['score_sum'].between(0, 200), 3, df['score_sum'])
#df['score_sum'] = np.where(df['score_sum'].between(201, 229), (3+22), df['score_sum'])
#df['score_sum'] = np.where(df['score_sum'].between(230, 239), (3+44), df['score_sum'])
#df['score_sum'] = np.where(df['score_sum'].between(240, 249), (3+66), df['score_sum'])
#df['z'] = np.where(df['score_sum'].between(250, 300), (3+88), df['score_sum'])

In [71]:
df.replace({0:0.001}, inplace = True)
df.fillna(0.001, inplace=True)

In [72]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor

    return interp_fn

In [73]:
for column in df[['score_x', 'score_meet_daily_targets', 'score_move_every_hour', 'score_recovery_time', 'score_stay_active', 'score_training_frequency', 'score_training_volume', 'non_wear', 'high', 'inactive', 'rest',  'low', 'medium', 'cal_total', 'daily_movement', 'score_y', 'score_activity_balance', 'score_hrv_balance', 'score_previous_day', 'score_previous_night', 'score_recovery_index', 'score_resting_hr', 'score_sleep_balance', 'score_temperature', 'awake', 'breath_average', 'deep', 'light', 'rem', 'hr_average', 'hr_lowest', 'score', 'score_alignment', 'score_deep', 'score_disturbances', 'score_efficiency', 'score_latency', 'score_rem', 'score_total']]:
    columnSeriesObj = df[column]
    col_min = df[column].min() 
    col_max = df[column].max() 
    col_list = df[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .03, .43) 
    col_interp = [scalar(x) for x in col_list] 
    df[(str(column) + str('_scale'))] = col_interp


In [74]:
col_list = df.columns.values.tolist()
# print(col_list)

In [75]:
# use to create x, y, z axes for root nodes
# np.linspace for x and y creates a 2D matrix of points, 
# you'll need the flatten or delete below to make it a list, not matrix.
# np.linspace(left value, right value, # of points)
# take number of rows in df from above .shape call. Choose grid sizing: 
# in my df, there were 9 rows, so 3x3 (length and width) is 9 total 
# positions. Then multiple that number by the spacing distance you want 
# between the root nodes of each glyph. In this case, I wanted 15 units. 
# So 0 to 30, with 3 points, yields positions of 0, 15, and 30.
# distance between each number
spacingx = 11
spacingy = 6.2

xxa = (7 * spacingx)/2
x = np.linspace(-(xxa), (xxa-1), 7)

yy = math.ceil((int(len(df.index)))/7)
yya = (yy*spacingy)/2
y = np.linspace(-(yya), (yya), yy)
x_1, y_1 = np.meshgrid(x, y)

extra = (yy*7)- (int(len(df.index)))

In [76]:
if extra == 0:
    x_1 = x_1.flatten()
else: 
    listextra = []
    for n in range(extra):
      listextra.append(n)   
    x_1= np.delete(x_1, listextra)

x_1.size

50

In [77]:
if extra == 0:
    y_1 = y_1.flatten()
else: 
    listextra = []
    for n in range(extra):
      listextra.append(n)   
    y_1= np.delete(y_1, listextra)
    
y_1.size

50

In [78]:
df['x'] = x_1
df['y'] = y_1
df['z'] = 3

In [79]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 16
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [80]:
#Leave this section as is, until 'numlevel1toroids'
# Open ANTz Node file for writing
fout = open(mn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(mt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 2

for index, row in df.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['x']
    translate_y = row['y']
    translate_z = row['z']
    scale_x = 1 
    scale_y = scale_x
    scale_z = .84
    ratio = .03
    line_width= 1
    geometry = 19
    topo = 6
    color_r = 151
    color_g = 193
    color_b = 201
    color_a = 255
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "Date." + str(row['day_end'])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    
    for j in range(numlevel1toroids):
        # scores data/ diamond
        if j == 0:
            id += 1
            data = id
            type = 5
            parent_id = level0objectid
            selected = 0
            branch_level = 1
            child_id = id
            translate_x = -30
            translate_y = 0
            translate_z = 0
            scale_x = .33
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 11
            topo = 3
            color_r = 161
            color_g = 150
            color_b = 214
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)
        
            level1objectid = id
            numlevel2toroids = 3
            #k are top 3 rods
            for k in range(numlevel2toroids):
                if k == 0:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level1objectid
                    selected = 0
                    branch_level = 2
                    child_id = id
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    scale_x = 3.38 
                    scale_y = scale_x
                    scale_z = 4.13
                    ratio = .18
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 158
                    color_g = 168
                    color_b = 186
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level2objectid = id
                   
                    # sleep score data
                    
                    numlevel3toroids = 8
                    
                    for m in range(numlevel3toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level2objectid
                        selected = 0
                        branch_level = 3
                        child_id = id
                        if m == 0:
                            translate_x = 45
                            color_r = 102
                            color_g = 0
                            color_b = 51
                            ratio = row['score_scale'] 
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 3
                            topo = 2
                            tagtext = "Sleep Score: " + str(row['score'])
                        elif m == 1:
                            translate_x = 0
                            color_r = 153
                            color_g = 0
                            color_b = 76
                            ratio = row['score_alignment_scale'] 
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - alignment: " + str(row['score_alignment'])
                        elif m == 2: 
                            translate_x = -30
                            color_r = 204
                            color_g = 0
                            color_b = 102
                            ratio = row['score_deep_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - deep: " + str(row['score_deep'])
                        elif m == 3:
                            translate_x = -60
                            color_r = 255
                            color_g = 0
                            color_b = 127
                            ratio = row['score_rem_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - rem: " + str(row['score_rem'])
                        elif m == 4:
                            translate_x = -90
                            color_r = 255
                            color_g = 51
                            color_b = 153
                            ratio = row['score_disturbances_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - disturbances: " + str(row['score_disturbances'])
                        elif m == 5:
                            translate_x = -120
                            color_r = 255
                            color_g = 102
                            color_b = 178
                            ratio = row['score_total_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - total time: " + str(row['score_total'])
                        elif m == 6:
                            translate_x = -150
                            color_r = 255
                            color_g = 153
                            color_b = 204
                            ratio = row['score_efficiency_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - efficiency: " + str(row['score_efficiency'])
                        elif m == 7: 
                            translate_x = -180
                            color_r = 255
                            color_g = 204
                            color_b = 229
                            ratio = row['score_latency_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep score - latency: " + str(row['score_latency'])
                        translate_y = 0
                        translate_z = 0
                        line_width= 1
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        
                        # Output to Tag file 
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                        
                            
                elif k == 1: 
                    id += 1
                    data = id
                    type = 5
                    parent_id = level1objectid
                    selected = 0
                    branch_level = 2
                    child_id = id
                    translate_x = -120
                    translate_y = 0
                    translate_z = 0
                    scale_x = 3.38
                    scale_y = scale_x
                    scale_z = 4.13
                    line_width= 1
                    ratio = .18
                    geometry = 19
                    topo = 6
                    color_r = 158
                    color_g = 168
                    color_b = 186
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level2objectid = id
                    #activity score data
                    numlevel3toroids = 7
                    for n in range(numlevel3toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level2objectid
                        selected = 0
                        branch_level = 3
                        child_id = id
                        if n == 0:
                            translate_x = 45
                            color_r = 51
                            color_g = 0
                            color_b = 102
                            ratio = row['score_x_scale'] 
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 3
                            topo = 2
                            tagtext = "Activity Score: " + str(row['score_x'])
                        elif n == 1:
                            translate_x = 0
                            color_r = 76
                            color_g = 0
                            color_b = 153
                            ratio = row['score_meet_daily_targets_scale'] 
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Activity score - hit daily targets: " + str(row['score_meet_daily_targets'])
                        elif n == 2: 
                            translate_x = -36
                            color_r = 102
                            color_g = 0
                            color_b = 204
                            ratio = row['score_move_every_hour_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Activity score - move each hour: " + str(row['score_move_every_hour'])
                        elif n == 3:
                            translate_x = -72
                            color_r = 127
                            color_g = 0
                            color_b = 255
                            ratio = row['score_recovery_time_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Activity score - recovery time: " + str(row['score_recovery_time'])
                        elif n == 4:
                            translate_x = -108
                            color_r = 153
                            color_g = 51
                            color_b = 255
                            ratio = row['score_stay_active_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Activity score - stay active: " + str(row['score_stay_active'])
                        elif n == 5:
                            translate_x = -144
                            color_r = 178
                            color_g = 102
                            color_b = 255
                            ratio = row['score_training_frequency_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            tagtext = "Activity score - training frequency: " + str(row['score_training_frequency'])
                            geometry = 7
                            topo = 3
                        elif n == 6:
                            translate_x = -180
                            color_r = 204
                            color_g = 153
                            color_b = 255
                            ratio = row['score_training_volume_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            tagtext = "Activity score - training volume: " + str(row['score_training_volume'])
                            geometry = 7
                            topo = 3
                        translate_y = 0
                        translate_z = 0
                        line_width= 1
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        
                        # Output to Tag file 
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                        
                        
                elif k == 2: 
                    id += 1
                    data = id
                    type = 5
                    parent_id = level1objectid
                    selected = 0
                    branch_level = 2
                    child_id = id
                    translate_x = 120
                    translate_y = 0
                    translate_z = 0
                    scale_x = 3.38 
                    scale_y = scale_x
                    scale_z = 4.13
                    line_width= 1
                    ratio = .18
                    geometry = 19
                    topo = 6
                    color_r = 158
                    color_g = 168
                    color_b = 186
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level2objectid = id
                    numlevel3toroids = 9
                    # readiness score data
                    for o in range(numlevel3toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level2objectid
                        selected = 0
                        branch_level = 3
                        child_id = id
                        if o == 0:
                            translate_x = -45
                            color_r = 0
                            color_g = 51
                            color_b = 102
                            ratio = row['score_y_scale'] 
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 3
                            topo = 2
                            tagtext = "Readiness Score: " + str(row['score_y'])
                        elif o == 1:
                            translate_x = 0
                            color_r = 0
                            color_g = 76
                            color_b = 153
                            ratio = row['score_activity_balance_scale'] 
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - activity balance: " + str(row['score_activity_balance'])
                        elif o == 2: 
                            translate_x = -26
                            color_r = 0
                            color_g = 102
                            color_b = 204
                            ratio = row['score_hrv_balance_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - hrv balance: " + str(row['score_hrv_balance'])
                        elif o == 3:
                            translate_x = -52
                            color_r = 0
                            color_g = 128
                            color_b = 255
                            ratio = row['score_previous_day_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - previous day: " + str(row['score_previous_day'])
                        elif o == 4:
                            translate_x = -78
                            color_r = 51
                            color_g = 153
                            color_b = 255
                            ratio = row['score_previous_night_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - previous night: " + str(row['score_previous_night'])
                        elif o == 5:
                            translate_x = -104
                            color_r = 102
                            color_g = 178
                            color_b = 255
                            ratio = row['score_recovery_index_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - recovery index: " + str(row['score_recovery_index'])
                        elif o == 6:
                            translate_x = -130
                            color_r = 153
                            color_g = 204
                            color_b = 255
                            ratio = row['score_resting_hr_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - resting hr: " + str(row['score_resting_hr'])
                        elif o == 7: 
                            translate_x = -156
                            color_r = 153
                            color_g = 204
                            color_b = 255
                            ratio = row['score_sleep_balance_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - sleep balance: " + str(row['score_sleep_balance'])
                        elif o == 8: 
                            translate_x = -180
                            color_r = 204
                            color_g = 229
                            color_b = 255
                            ratio = row['score_temperature_scale']
                            scale_x = 3.8
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Readiness score - temperature: " + str(row['score_temperature'])
                        translate_y = 0
                        translate_z = 0
                        line_width= 1
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        
                        # Output to Tag file 
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                        
                            
        elif j == 1:
            # additional variables
            id += 1
            data = id
            type = 5
            parent_id = level0objectid
            selected = 0
            branch_level = 1
            child_id = id
            translate_x = -130
            translate_y = 0
            translate_z = 0
            scale_x = .33 
            scale_y = scale_x
            scale_z = scale_x
            line_width= 1
            geometry = 9
            topo = 3
            color_r = 73
            color_g = 126
            color_b = 184
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)
        
            level1objectid = id
            
            for l in range(numlevel2toroids):
                if l == 0:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level1objectid
                    selected = 0
                    branch_level = 2
                    child_id = id
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    scale_x = 3.38 
                    scale_y = scale_x
                    scale_z = 4.13
                    ratio = .18
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 158
                    color_g = 168
                    color_b = 186
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level2objectid = id
                    
                    # sleep additional data variables
                    numlevel3toroids = 7
                    for p in range(numlevel3toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level2objectid
                        selected = 0
                        branch_level = 3
                        child_id = id
                        if p == 0:
                            translate_x = 45
                            color_r = 102
                            color_g = 0
                            color_b = 51
                            ratio = row['awake_scale'] 
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep time awake: " + str(row['awake'])
                        elif p == 1:
                            translate_x = 0
                            color_r = 153
                            color_g = 0
                            color_b = 76
                            ratio = row['light_scale'] 
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep time light: " + str(row['light'])
                        elif p == 2: 
                            translate_x = -36
                            color_r = 204
                            color_g = 0
                            color_b = 102
                            ratio = row['deep_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep time deep: " + str(row['deep'])
                        elif p == 3:
                            translate_x = -72
                            color_r = 255
                            color_g = 0
                            color_b = 127
                            ratio = row['rem_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Sleep time rem: " + str(row['rem'])
                        elif p == 4:
                            translate_x = -108
                            color_r = 255
                            color_g = 51
                            color_b = 153
                            ratio = row['breath_average_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Average breath while asleep: " + str(row['breath_average'])
                        elif p == 5:
                            translate_x = -144
                            color_r = 255
                            color_g = 102
                            color_b = 178
                            ratio = row['hr_average_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Average HR while asleep: " + str(row['hr_average'])
                        elif p == 6:
                            translate_x = -180
                            color_r = 255
                            color_g = 153
                            color_b = 204
                            ratio = row['hr_lowest_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Lowest HR while asleep: " + str(row['hr_lowest'])
                        translate_y = 0
                        translate_z = 0
                        line_width= 1
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        
                        # Output to Tag file 
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                            
                elif l == 1:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level1objectid
                    selected = 0
                    branch_level = 2
                    child_id = id
                    translate_x = 120
                    translate_y = 0
                    translate_z = 0
                    scale_x = 3.38 
                    scale_y = scale_x
                    scale_z = 4.13
                    ratio = .18
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 158
                    color_g = 168
                    color_b = 186
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    level2objectid = id
                    
                    # activity additional data variables
                    numlevel3toroids = 8
                    for q in range(numlevel3toroids):
                        id += 1
                        data = id
                        type = 5
                        parent_id = level2objectid
                        selected = 0
                        branch_level = 3
                        child_id = id
                        if q == 0:
                            translate_x = 45
                            color_r = 51
                            color_g = 0
                            color_b = 102
                            ratio = row['non_wear_scale'] 
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Time ring not worn: " + str(row['non_wear'])
                        elif q == 1:
                            translate_x = 0
                            color_r = 76
                            color_g = 0
                            color_b = 153
                            ratio = row['rest_scale'] 
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Time at rest: " + str(row['rest'])
                        elif q == 2: 
                            translate_x = -30
                            color_r = 102
                            color_g = 0
                            color_b = 204
                            ratio = row['inactive_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Time inactive: " + str(row['inactive'])
                        elif q == 3:
                            translate_x = -60
                            color_r = 127
                            color_g = 0
                            color_b = 255
                            ratio = row['low_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Time with low activity: " + str(row['low'])
                        elif q == 4:
                            translate_x = -90
                            color_r = 153
                            color_g = 51
                            color_b = 255
                            ratio = row['medium_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Time with medium activity: " + str(row['medium'])
                        elif q == 5:
                            translate_x = -120
                            color_r = 178
                            color_g = 102
                            color_b = 255
                            ratio = row['high_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Time with high activity: " + str(row['high'])
                        elif q == 6:
                            translate_x = -150
                            color_r = 204
                            color_g = 153
                            color_b = 255
                            ratio = row['daily_movement_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Daily Movement Total: " + str(row['daily_movement'])
                        elif q == 7:
                            translate_x = -180
                            color_r = 229
                            color_g = 204
                            color_b = 255
                            ratio = row['cal_total_scale']
                            scale_x = 3.33
                            scale_y = scale_x
                            scale_z = scale_x
                            geometry = 7
                            topo = 3
                            tagtext = "Total Calories Burned: " + str(row['cal_total'])
                        translate_y = 0
                        translate_z = 0
                        line_width= 1
                        color_a = 255
                        record_id = id 
                        texture_id= 0

                        # Creat output string and write to ANTz Node File
                        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                        fout.write(outputstring)
                        
                        # Output to Tag file 
                        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                        ftag.write(tagstring)
                        taginc += 1
                            
                            
                elif l == 2:
                    id += 1
                    data = id
                    type = 5
                    parent_id = level1objectid
                    selected = 0
                    branch_level = 2
                    child_id = id
                    translate_x = -120
                    translate_y = 0
                    translate_z = 0
                    scale_x = 3.38 
                    scale_y = scale_x
                    scale_z = 4.13
                    ratio = .18
                    line_width= 1
                    geometry = 19
                    topo = 6
                    color_r = 158
                    color_g = 168
                    color_b = 186
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)
                    
                    
                    
                    